In [1]:
#imports
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt

import re, ast
from scipy.sparse import csr_matrix, hstack, vstack, issparse

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [11]:
df = pd.read_csv('../data/prep.csv')
df.columns

Index(['parent_asin', 'price', 'rating', 'user_id', 'merged_text'], dtype='object')

In [12]:
item_df = (
    df.groupby('parent_asin')
    .agg(
        price = ('price', 'mean'),
        rating = ('rating', 'mean'),
        num_ratings = ('rating', 'count'),
        text = ('merged_text', 'first'),
    )
)

item_df.head()

,price,rating,num_ratings,text
parent_asin,,,,
0007922582,51.0,2.000000,1,The Sneetches and Other Stories Too small w...
0008288194,14.0,3.000000,1,The Creativity Code Video Games PC Gam...
0028179714,15.0,4.666667,3,The Autobiography of Miss Jane Pittman and Rel...
0060501960,8.0,4.470588,17,Presidents Day Video Games PC Games ...
0063052164,20.0,5.000000,4,Stranger Planet AUTOGRAPHED SIGNED BOOK Vi...


In [21]:
# tf-idf transformation
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = tfidf.fit_transform(item_df['text'])
tfidf_matrix.shape

(137249, 5000)

In [15]:
# standard scaling for numeric features
numeric_features = ['price']
scaler = StandardScaler()

numeric_scaled = scaler.fit_transform(item_df[numeric_features])

In [26]:
# Creating matrices
numeric_sparse = csr_matrix(np.nan_to_num(numeric_scaled))
hybrid_matrix = hstack([tfidf_matrix, numeric_sparse])

tfidf_index = df['parent_asin'].reset_index(drop=True)
hybrid_matrix.shape

(137249, 5001)